In [2]:
import subprocess
import numpy as np
import time

In [1]:
import os

In [ ]:
def evaluate_brotli(quant_embed, quant_ckt, sym_bit_dict, final_size, full_data_length): 
  quant_v_list = quant_embed['quant'].flatten().tolist()
  tmin_scale_len = quant_embed['min'].nelement() + quant_embed['scale'].nelement()
  for k, layer_wt in quant_ckt.items():
      quant_v_list.extend(layer_wt['quant'].flatten().tolist())
      tmin_scale_len += layer_wt['min'].nelement() + layer_wt['scale'].nelement()

  # get the element name and its frequency
  unique, counts = np.unique(quant_v_list, return_counts=True)
  num_freq = dict(zip(unique, counts))

  comp_encode_start_time = time.time()
  subprocess.run(["brotli", "-q", "11", "-o", "compressed_quant_vid.pth.br", "quant_vid.pth"])
  comp_encode_end_time = time.time()
  delta_encode_comp = comp_encode_end_time - comp_encode_start_time

  decomp_decode_start_time = time.time()
  subprocess.run(["brotli", "-d", "-o", "quant_vid.pth", "compressed_quant_vid.pth.br"])
  decomp_decode_end_time = time.time()
  delta_decode_decomp = decomp_decode_end_time - decomp_decode_start_time

  # total bits for quantized embed + model weights
  # total_bits = 0
  # for num, freq in num_freq.items():
  #     total_bits += freq * sym_bit_dict[num]
  # bits_per_param = total_bits / len(quant_v_list)

  # # including the overhead for min and scale storage, 
  # total_bits += tmin_scale_len * 16               #(16bits for float16)
  # full_bits_per_param = total_bits / len(quant_v_list)

  filename = 'compressed_quant_vid.pth.br'
  total_bits = os.path.getsize(filename) * 8

  # bits per pixel
  total_bpp = total_bits /final_size /full_data_length
  print(f'bits per pixel: {round(total_bpp, 4)}')
  # import pdb; pdb.set_trace; from IPython import embed; embed()     
  return delta_encode_comp, delta_decode_decomp, total_bpp
